In [27]:
import pandas as pd

# Đọc tệp CSV
full_train = pd.read_csv('C:/python/full_train_1.csv')

# Cấu hình pandas để hiển thị tất cả các dòng
pd.set_option('display.max_rows', None)

# In ra tất cả các dòng của DataFrame
print(full_train)



IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [29]:
import pandas as pd

# Đọc tệp CSV
full_train = pd.read_csv('C:/python/full_train_1.csv')

# In ra 20 dòng đầu tiên
print(full_train.head(20))

# In ra tổng số dòng của DataFrame
print(f'Tổng số dòng: {full_train.shape[0]}')


    Unnamed: 0                                            Comment  Rating  \
0          0.0                                      Món ăn cực tệ     0.0   
1          1.0  Mình thấy mọi người review tích cực nhưng thực...     0.0   
2          2.0  Chắc với sinh viên thì ai cũng quen thuộc với ...     1.0   
3          3.0  Nhiều hôm thèm lẩu riêu mà chỉ có một mình thì...     1.0   
4          4.0  Không gian sạch sẽ.quán mới lên ra món hơi chậ...     1.0   
5          5.0                  Nhất định sẽ ủng hộ quán dài dài…     1.0   
6          6.0  Lang thang với cái bụng đói.đứng chờ tàu thì n...     1.0   
7          7.0  Cá thái quá to, rán cháy, cứng, khô.\nNước dùn...     0.0   
8          8.0  Đói quá vội ăn nên ko chụp đc cái ảnh tử tế. P...     1.0   
9          9.0  Mình vẫn k hiểu vì sao quán mở được đến tận 7-...     0.0   
10        10.0  Mình ăn hai món đặc trưng nhất của thái là lẩu...     0.0   
11        11.0  Nhìn chung đồ ăn khá hợp vị. Nấm chiên cũng rấ...     1.0   

In [31]:
# Xóa các cột không cần thiết, chỉ giữ lại 'Comment' và 'Rating'
full_train = full_train[['Comment', 'Rating']]

# Thêm cột 'Sentiment' dựa trên giá trị của cột 'Rating' (0: negative, 1: positive)
full_train['Sentiment'] = full_train['Rating'].apply(lambda x: 1 if x > 0 else 0)

# Thêm cột 'Sentiment_Label' với giá trị 'positive' cho 1 và 'negative' cho 0
full_train['Sentiment_Label'] = full_train['Sentiment'].apply(lambda x: 'positive' if x == 1 else 'negative')

# Xóa cột 'Rating' vì đã có thông tin trong cột 'Sentiment' và 'Sentiment_Label'
full_train = full_train.drop(columns=['Rating'])

# In ra 20 dòng đầu tiên sau khi chỉnh sửa
print(full_train.head(20))

# In ra tổng số dòng và cột
print(f'Tổng số dòng: {full_train.shape[0]}, Tổng số cột: {full_train.shape[1]}')

                                              Comment  Sentiment  \
0                                       Món ăn cực tệ          0   
1   Mình thấy mọi người review tích cực nhưng thực...          0   
2   Chắc với sinh viên thì ai cũng quen thuộc với ...          1   
3   Nhiều hôm thèm lẩu riêu mà chỉ có một mình thì...          1   
4   Không gian sạch sẽ.quán mới lên ra món hơi chậ...          1   
5                   Nhất định sẽ ủng hộ quán dài dài…          1   
6   Lang thang với cái bụng đói.đứng chờ tàu thì n...          1   
7   Cá thái quá to, rán cháy, cứng, khô.\nNước dùn...          0   
8   Đói quá vội ăn nên ko chụp đc cái ảnh tử tế. P...          1   
9   Mình vẫn k hiểu vì sao quán mở được đến tận 7-...          0   
10  Mình ăn hai món đặc trưng nhất của thái là lẩu...          0   
11  Nhìn chung đồ ăn khá hợp vị. Nấm chiên cũng rấ...          1   
12  Chị bán hàng phục vụ nhiệt tình lại còn luôn v...          1   
13  Mình bị nghiện món này nên hôm nào dù nhà xa

In [33]:
import re
from underthesea import word_tokenize

# Danh sách từ dừng (stopwords) bằng tiếng Việt (Cần có hoặc bạn có thể tự tạo)
vn_stopwords = set(["và", "hoặc", "theo", "của", "là", "lúc", "trong", "tại", "để", "có", "mà", "với", "để", "bằng", "từ", "một", "đi", "sẽ", "cũng", "không", "như"])

# Hàm loại bỏ các từ dừng (stopwords)
def remove_stopword(review):
    filtered = ' '.join([word for word in review.split() if word not in vn_stopwords])
    return filtered

# Định nghĩa bản đồ các từ viết tắt hoặc sai chính tả
correct_mapping = {
    "ship": "vận chuyển",
    "shop": "cửa hàng",
    "m": "mình",
    "mik": "mình",
    "ko": "không",
    "k": " không ",
    "kh": "không",
    "khong": "không",
    "kg": "không",
    "khg": "không",
    "tl": "trả lời",
    "r": "rồi",
    "fb": "mạng xã hội", # facebook
    "face": "mạng xã hội",
    "thanks": "cảm ơn",
    "thank": "cảm ơn",
    "tks": "cảm ơn",
    "tk": "cảm ơn",
    "ok": "tốt",
    "dc": "được",
    "vs": "với",
    "đt": "điện thoại",
    "thjk": "thích",
    "qá": "quá",
    "trể": "trễ",
    "bgjo": "bao giờ",
    "good": "tốt",
    "bh": "bây giờ",
    "sale": "giảm giá",
    "ntn": "như thế này",
    "vote": "đánh giá tốt",
    "ms": "mới",
    "hnay": "hôm nay",
    "kute": "dễ thương",
    "bik": "biết",
    "od": "gọi món",
    "mn": "mọi người",
    "c": "chị",
    "đc": "được",
    "uk": "ừ",
    "t": "tôi",
    "tt": "thứ tự",
    "gj": "gì",
    "j": "gì",
    "đx": "được",
    "m": "mày",
    "zậy": "vậy",
    "wa": "qua",
    "zui": "vui",
    "thik": "thích",
    "add": "thêm",
    "pko": "phải không",
    "cmt": "bình luận",
    "dt": "dễ thương",
    "ib": "inbox",
    "klq": "không liên quan",
    "nx": "nhận xét",
    "rep": "trả lời",
    "dj": "đi",
    "mog": "mong",
    "bít": "biết",
    "nc": "nước",
    "lun": "luôn",
    "hiu": "hiểu",
    "rui": "rồi",
    "thui": "thôi",
    "view": "phong cảnh",
    "đg": "đang",
    "h": "giờ",
    "zòn": "giòn",
    "cx": "cũng",
    "kbiet": "không biết",
    "đ": "đéo",
    "mk": "mình",
    "trc": "trước",
    "bùn": "buồn",
    "iu": "yêu",
    "vs": "với",
    "lua": "lừa",
    "b": "bạn"
}

# Hàm kiểm tra và thay thế từ viết tắt hoặc sai chính tả
def check_acronyms(w):
    if w in correct_mapping:
        return correct_mapping[w]
    else:
        return w

# Hàm thay thế các từ trong câu theo bản đồ
def mapping(review):
    word = review.split()
    word = map(check_acronyms, word)
    word = ' '.join(word)
    return word

# Hàm tiền xử lý dữ liệu
def preprocess_data(df, column_name='Review'):
    # Chuyển tất cả ký tự về chữ thường
    df.loc[:, column_name] = df[column_name].apply(lambda x: str.lower(x))
    
    # Loại bỏ ký tự thừa như \n, dấu cách thừa
    df.loc[:, column_name] = df[column_name].apply(lambda x: x.strip())
    
    # Loại bỏ các con số
    df.loc[:, column_name] = df[column_name].apply(lambda x: re.sub('\d', '', x))
    
    # Loại bỏ các ký tự đặc biệt
    df.loc[:, column_name] = df[column_name].apply(lambda x: re.sub('\W+', ' ', x))
    
    # Loại bỏ stopwords (từ dừng)
    df.loc[:, column_name] = df[column_name].apply(lambda x: remove_stopword(x))
    
    # Chuẩn hóa từ ngữ (viết tắt, sai chính tả)
    df.loc[:, column_name] = df[column_name].apply(lambda x: mapping(x))
    
    # Xóa các dòng trống
    df = df[df[column_name].notnull() & (df[column_name] != '')]
    
    return df

# Giả sử df là DataFrame chứa dữ liệu bình luận, và 'Review' là tên cột chứa các bình luận
df = preprocess_data(df, column_name='Review')

# In kết quả
print(df.head())


<>:119: SyntaxWarning: invalid escape sequence '\d'
<>:122: SyntaxWarning: invalid escape sequence '\W'
<>:119: SyntaxWarning: invalid escape sequence '\d'
<>:122: SyntaxWarning: invalid escape sequence '\W'
C:\Users\ducan\AppData\Local\Temp\ipykernel_2460\3545640624.py:119: SyntaxWarning: invalid escape sequence '\d'
  df.loc[:, column_name] = df[column_name].apply(lambda x: re.sub('\d', '', x))
C:\Users\ducan\AppData\Local\Temp\ipykernel_2460\3545640624.py:122: SyntaxWarning: invalid escape sequence '\W'
  df.loc[:, column_name] = df[column_name].apply(lambda x: re.sub('\W+', ' ', x))
C:\Users\ducan\AppData\Local\Temp\ipykernel_2460\3545640624.py:119: SyntaxWarning: invalid escape sequence '\d'
  df.loc[:, column_name] = df[column_name].apply(lambda x: re.sub('\d', '', x))
C:\Users\ducan\AppData\Local\Temp\ipykernel_2460\3545640624.py:122: SyntaxWarning: invalid escape sequence '\W'
  df.loc[:, column_name] = df[column_name].apply(lambda x: re.sub('\W+', ' ', x))


ModuleNotFoundError: No module named 'underthesea'

In [35]:

import re
from underthesea import word_tokenize

# Danh sách từ dừng (stopwords) bằng tiếng Việt (Cần có hoặc bạn có thể tự tạo)
vn_stopwords = set(["và", "hoặc", "theo", "của", "là", "lúc", "trong", "tại", "để", "có", "mà", "với", "để", "bằng", "từ", "một", "đi", "sẽ", "cũng", "không", "như"])

# Hàm loại bỏ các từ dừng (stopwords)
def remove_stopword(review):
    filtered = ' '.join([word for word in review.split() if word not in vn_stopwords])
    return filtered

# Định nghĩa bản đồ các từ viết tắt hoặc sai chính tả
correct_mapping = {
    "ship": "vận chuyển",
    "shop": "cửa hàng",
    "m": "mình",
    "mik": "mình",
    "ko": "không",
    "k": " không ",
    "kh": "không",
    "khong": "không",
    "kg": "không",
    "khg": "không",
    "tl": "trả lời",
    "r": "rồi",
    "fb": "mạng xã hội", # facebook
    "face": "mạng xã hội",
    "thanks": "cảm ơn",
    "thank": "cảm ơn",
    "tks": "cảm ơn",
    "tk": "cảm ơn",
    "ok": "tốt",
    "dc": "được",
    "vs": "với",
    "đt": "điện thoại",
    "thjk": "thích",
    "qá": "quá",
    "trể": "trễ",
    "bgjo": "bao giờ",
    "good": "tốt",
    "bh": "bây giờ",
    "sale": "giảm giá",
    "ntn": "như thế này",
    "vote": "đánh giá tốt",
    "ms": "mới",
    "hnay": "hôm nay",
    "kute": "dễ thương",
    "bik": "biết",
    "od": "gọi món",
    "mn": "mọi người",
    "c": "chị",
    "đc": "được",
    "uk": "ừ",
    "t": "tôi",
    "tt": "thứ tự",
    "gj": "gì",
    "j": "gì",
    "đx": "được",
    "m": "mày",
    "zậy": "vậy",
    "wa": "qua",
    "zui": "vui",
    "thik": "thích",
    "add": "thêm",
    "pko": "phải không",
    "cmt": "bình luận",
    "dt": "dễ thương",
    "ib": "inbox",
    "klq": "không liên quan",
    "nx": "nhận xét",
    "rep": "trả lời",
    "dj": "đi",
    "mog": "mong",
    "bít": "biết",
    "nc": "nước",
    "lun": "luôn",
    "hiu": "hiểu",
    "rui": "rồi",
    "thui": "thôi",
    "view": "phong cảnh",
    "đg": "đang",
    "h": "giờ",
    "zòn": "giòn",
    "cx": "cũng",
    "kbiet": "không biết",
    "đ": "đéo",
    "mk": "mình",
    "trc": "trước",
    "bùn": "buồn",
    "iu": "yêu",
    "vs": "với",
    "lua": "lừa",
    "b": "bạn"
}

# Hàm kiểm tra và thay thế từ viết tắt hoặc sai chính tả
def check_acronyms(w):
    if w in correct_mapping:
        return correct_mapping[w]
    else:
        return w

# Hàm thay thế các từ trong câu theo bản đồ
def mapping(review):
    word = review.split()
    word = map(check_acronyms, word)
    word = ' '.join(word)
    return word

# Hàm tiền xử lý dữ liệu
def preprocess_data(df, column_name='Review'):
    # Chuyển tất cả ký tự về chữ thường
    df.loc[:, column_name] = df[column_name].apply(lambda x: str.lower(x))
    
    # Loại bỏ ký tự thừa như \n, dấu cách thừa
    df.loc[:, column_name] = df[column_name].apply(lambda x: x.strip())
    
    # Loại bỏ các con số
    df.loc[:, column_name] = df[column_name].apply(lambda x: re.sub('\d', '', x))
    
    # Loại bỏ các ký tự đặc biệt
    df.loc[:, column_name] = df[column_name].apply(lambda x: re.sub('\W+', ' ', x))
    
    # Loại bỏ stopwords (từ dừng)
    df.loc[:, column_name] = df[column_name].apply(lambda x: remove_stopword(x))
    
    # Chuẩn hóa từ ngữ (viết tắt, sai chính tả)
    df.loc[:, column_name] = df[column_name].apply(lambda x: mapping(x))
    
    # Xóa các dòng trống
    df = df[df[column_name].notnull() & (df[column_name] != '')]
    
    return df

# Giả sử df là DataFrame chứa dữ liệu bình luận, và 'Review' là tên cột chứa các bình luận
df = preprocess_data(df, column_name='Review')

# In kết quả
print(df.head())


<>:119: SyntaxWarning: invalid escape sequence '\d'
<>:122: SyntaxWarning: invalid escape sequence '\W'
<>:119: SyntaxWarning: invalid escape sequence '\d'
<>:122: SyntaxWarning: invalid escape sequence '\W'
C:\Users\ducan\AppData\Local\Temp\ipykernel_2460\1347729575.py:119: SyntaxWarning: invalid escape sequence '\d'
  df.loc[:, column_name] = df[column_name].apply(lambda x: re.sub('\d', '', x))
C:\Users\ducan\AppData\Local\Temp\ipykernel_2460\1347729575.py:122: SyntaxWarning: invalid escape sequence '\W'
  df.loc[:, column_name] = df[column_name].apply(lambda x: re.sub('\W+', ' ', x))
C:\Users\ducan\AppData\Local\Temp\ipykernel_2460\1347729575.py:119: SyntaxWarning: invalid escape sequence '\d'
  df.loc[:, column_name] = df[column_name].apply(lambda x: re.sub('\d', '', x))
C:\Users\ducan\AppData\Local\Temp\ipykernel_2460\1347729575.py:122: SyntaxWarning: invalid escape sequence '\W'
  df.loc[:, column_name] = df[column_name].apply(lambda x: re.sub('\W+', ' ', x))


ModuleNotFoundError: No module named 'underthesea'